# Analyse des occupations et des profils des acteurs à l'époque moderne et contemporaine 

Dans ce carnet la méthode d'analyse de correspondances multiples est appliquée afin de mettre en évidence les profils des acteurs dans le champ scientifique de l'astronomie et de la physique.

Pour des raisons d'effectifs et de cohérence culturelle l'analyse est effectuée sur les époques moderne (avant 1801) et contemporaine (après 1801) séparément

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE, XML, RDFXML

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:

import seaborn as sns

In [ ]:
### Librairies déjà installées avec Python
import pprint
import sqlite3 as sql

import pprint
import csv
import sys


import time
import datetime
from dateutil import parser

from importlib import reload
from shutil import copyfile


In [ ]:
### https://mwouts.github.io/itables/quick_start.html

from itables import init_notebook_mode, show

init_notebook_mode(all_interactive=False)

In [ ]:
from fanalysis.ca import CA 
from fanalysis.mca import MCA

In [ ]:
### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
sys.path.insert(0, '../..')


import sparql_functions as spqf

## Importer les données à analyser

On exécute une requête SQL sur la base de données qui recompose les données et effectue un premier codage.

In [ ]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/astronomers_import.db')
cn

In [ ]:
### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
sys.path.insert(0, '../..')


import sparql_functions as spqf

In [ ]:
qr = """
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX franzOption_defaultDatasetBehavior: <franz:rdf>

SELECT DISTINCT ?item ?occupation ?occupationLabel ?occupation_1 ?occupation_1Label
WHERE
  {GRAPH <https://github.com/Sciences-historiques-numeriques/astronomers/blob/main/graphs/wikidata-imported-data.md>
     {
      ?item wdt:P106 ?occupation.
      ?item wdt:P106 ?occupation_1.
      FILTER (str(?occupation) < str(?occupation_1))
      ?occupation rdfs:label ?occupationLabel.
      ?occupation_1 rdfs:label ?occupation_1Label.
     } 

      }
"""

In [ ]:
### Vérifier que les données ont été importées correctement
cur = cn.cursor()
l = cur.execute(qr).fetchall()

In [ ]:
### Inspection des premières lignes
print(len(l))
l[:3]

In [ ]:
df_orig = pd.DataFrame(l, columns=['uri', 'nom', 'gender', 'annee_nais',
                     'eff_occupations', 'liste_occup', 'origine_geog'])
df_orig.info()

In [ ]:
### Inspection des données 
show(df_orig[df_orig.annee_nais < '1901' ].sort_values(by='annee_nais', ascending=True), 
    scrollY="300px",
     scrollCollapse=True, paging=False, layout={"topEnd": None}, 
     showIndex=False, column_filters="header", columnDefs=[{"className": "dt-left", "targets": "_all"}],
     # https://mwouts.github.io/itables/downsampling.html
       maxBytes=0)

In [ ]:
### Distribution du nombre d'occupations par personne:
# combien de personnes ont plus qu'une occupation?
# Période contemporaine

dfs = df_orig[(df_orig.annee_nais > '1800') & (df_orig.annee_nais < '1981') ].groupby(by='eff_occupations')\
    .size()
print('Total:', sum(dfs))
dfs

In [ ]:
### Distribution du nombre d'occupations par personne
# Période moderne

dfs = df_orig[(df_orig.annee_nais < '1801') ].groupby(by='eff_occupations')\
    .size()
print('Total:', sum(dfs))
dfs

## Codage des occupations

### Création de la fonction de codage

On choisit parmi les modalités de la variable trois catégories dont le sens est différent:
* la première est la catégorie principale, celle qui a permis de constituer la population
* la deuxième est plus liée à la notion d'activité et non de domaine scientifique
* la troisième, comme la première, relève du domaine scientifique


À noter que l'ordre des disciplines compte: le fonction qu'on va construire utilise toujours la première modalité rencontrée pour chaque catégorie, et cesse ensuite de chercher.


Dans le choix de l'ordre on a choisi ici de mettre en premier les modalités rares ou plus intéressantes dans une logique de domaine d'études mais on pourrait faire autrement et on consruit ici __un modèle de codage__ qui n'est pas absolu et qu'il faudra expliciter dans le rendu du travail car il influence les résultats.

In [ ]:
# Disciplines principales, disc_princ
a = ['Astrologie', 'Astronomie', 'Physique']

# Activités, activites
b = ['Clerc', 'Ingenieur', 'Chercheur',  'Politique_droit', 'Economie','Enseignant']

# Autres disciplines, disc_autres
c = ['Medecine', 'Philosophie', 'Lettres', 'Arts',  'SHS', 'Pharmacie_chimie', 'Autres_sciences','Mathematiques']


In [ ]:
### Ajout de colonnes supplémentaires sur toute la table

## Dans la fonction, on met la liste à coder (l) et les listes des objets en argument

def coder(l, a, b, c):
    
    ll = []

    ## si il n'y a pas d'autres disciplines
    # complète avec disciplines principales additionnelles
    d = c + a

    for e in l:

        e = list(e)
        
        o1 = ''
        for el in a:
            # dès que la variable o1 est remplie sort de la boucle
            if el in e[5] and len(o1) == 0:
                o1 = el
                e.append('d1_'+ el)   
        if not(len(o1) > 0):
            e.append('d1_nr')

        o2 = ''
        for el in b:
            if el in e[5] and len(o2) == 0:
                o2 = el
                e.append('d2_'+ o2)
        if not(len(o2) > 0):
            e.append('d2_nr')


        o3 = ''
        for el in d:
            if el in e[5] and len(o3) == 0\
                    and el != o1:
                o3 = el
                e.append('d3_'+ o3)
        if not(len(o3) > 0):
            e.append('d3_nr')

        o4 = ''
        for el in d:
            if len(o4) == 0 and el in e[5]\
                    and el != o3 and el != o1:
                o4 = el
                e.append('d4_'+ o4)
        if not(len(o4) > 0):
            e.append('d4_nr')

        ll.append(e)

    return ll

In [ ]:
ll = coder(l, a, b, c)

In [ ]:
pprint.pprint(ll[103:110])

In [ ]:
len(l),len(ll), ll[301:305]

In [ ]:
# La deuxième variable (a1) est considérée comme une activité et non une occupation
df_ll = pd.DataFrame(ll, columns=['uri', 'nom', 'genre', 'annee_nais',
                     'eff_occupations', 'liste_occup', 'origine_geog', 'o1', 'a1', 'o2', 'o3'])
df_ll.info()

In [ ]:
df_ll.groupby(by='o1').size().sort_values(ascending=False)

In [ ]:
df_ll.groupby(by='a1').size().sort_values(ascending=False)

In [ ]:
df_ll[df_ll.o3 != 'd4_nr'].groupby(by='a1').size().sort_values(ascending=False)

In [ ]:
df_ll.groupby(by='o2').size().sort_values(ascending=False)

In [ ]:
df_ll.groupby(by='o3').size().sort_values(ascending=False)

### Codage par période de cinquante ans

In [ ]:
### On transforme le type de valeur de la colonne BirthYear
# de objet à entier
df_ll['annee_nais'] = df_ll['annee_nais'].astype(int)

In [ ]:
### Créer une copie indépendante du DataFrame (attention aux copies de variables qui sont seulement des alias) 
cdf_p = df_ll[df_ll['annee_nais'] < 1981].copy(deep=True)
cdf_p.head(3)

In [ ]:
### Année minimale et maximale dans la population
min(cdf_p['annee_nais']), max(cdf_p['annee_nais'])

In [ ]:
### Créer une liste d'années pour séparer en périodes de 25 ans
# noter que la dernière année sera exclue, 
# elle donc doit être supérieure à la valeur maximale 
l_50 = list(range(1351, 2002, 50))
l_50[:5],l_50[-5:]

In [ ]:
### fonction pd.cut : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html
# On ajoute une nouvelle colonne qui contient la période sur la base de la liste précédente
# et de la valeur de l'année

cdf_p['periodes'] = pd.cut(cdf_p['annee_nais'], l_50, right=False)

### Transformer le code ajouté pour qu'il soit plus lisible
# noter qu'on a arrondi les valeurs
cdf_p['periodes'] = cdf_p['periodes'].apply(lambda x : str(int(x.left))+'-'+ str(int(x.right)-1))

# Inspection
cdf_p.head(3)

In [ ]:
cdf_p[cdf_p.annee_nais == 1980][:3]

In [ ]:
### compter les naissances par périodes de 50 ans
ax = cdf_p.groupby(by='periodes', observed=True).size().plot(kind='bar',rot=60, fontsize=8, figsize=(16,8))
ax.bar_label(ax.containers[0], fontsize=12)

plt.ylabel('Effectif')
plt.xlabel('Périodes')
plt.title('Naissances par périodes de cinquante ans')
plt.show()

## MCA de la période avant 1801

### Tableau à analyser

Étant donné qu'on a observé de grandes différences d'effectifs pour les différentes périodes, on sépare deux périodes: une avant 1801 (appelée moderne), l'autre à partir de cette date (appelée contemporaine). Et on les analyse séparément.


On explore d'abord la distribution des différentes variables pour la première période.



In [ ]:
### On notera l'effectif trop important des individus pour lesquels cette variable
# n'est pas renseignée. Mieux vaudra ne pas l'utiliser
cdf_p[cdf_p.annee_nais < 1801].groupby(by='o3').size().sort_values(ascending=False)

In [ ]:
df_d3 = cdf_p[cdf_p.annee_nais < 1801].groupby(by='o2').size().sort_values(ascending=False)
df_d3

In [ ]:
## exclure les modalités moins fréquentes
l_d3 = df_d3.index.to_list()[:-2]
print(l_d3)

In [ ]:
df_d2 = cdf_p[cdf_p.annee_nais < 1801].groupby(by='a1').size().sort_values(ascending=False)
df_d2

In [ ]:
## exclure les modalités moins fréquentes
l_d2 = df_d2.index.to_list()[:-1]
print(l_d2)

In [ ]:
cdf_p[cdf_p.annee_nais < 1801].groupby(by='o1').size().sort_values(ascending=False)

In [ ]:
cdf_p[cdf_p.annee_nais < 1801].groupby(by='periodes', observed=True).size().sort_values(ascending=False)

In [ ]:
cdf_og = cdf_p[cdf_p.annee_nais < 1801].groupby(by='origine_geog', observed=True).size().sort_values(ascending=False)
cdf_og

In [ ]:
## exclure les modalités moins fréquentes
l_og = cdf_og.index.to_list()[:-3]
print(l_og)

### Préparer le tableau à analyser

Il est nécessaire de recoder le tableau pour ne pas avoir des variables avec modalités vides: on utilise donc un nouveau tableau avec les individus nés avant 1801.


Aussi, si on n'enlève pas les modalités les moins fréquentes (cf. distributions ci-dessus) elles impactent massivement l'analyse factorielle des correspondances multiples, car les modalités les moins fréquentes apportent le plus de variation. On doit donc inspecter manuellement les profils rares mais les exclure de l'ACM pour mettre en évidence des structures plus importantes.


Le bon dosage dépend des questions de recherche.

In [ ]:
### Créer une copie indépendante du DataFrame 
# tout en éliminant les modalités peu fréquentes 
cdf_mod = df_ll[(df_ll['annee_nais'] < 1801)\
                & (df_ll.origine_geog.isin(l_og))\
                & (df_ll.a1.isin(l_d2))\
                & (df_ll.o2.isin(l_d3))]\
                    .copy(deep=True).reset_index(names='orig_index')
print(len(cdf_mod))
cdf_mod.head(3)

In [ ]:
max(cdf_mod.index)

In [ ]:
### Année minimale et maximale dans la population
min(cdf_mod['annee_nais']), max(cdf_mod['annee_nais'])

In [ ]:
### Créer une liste d'années pour séparer en périodes de 25 ans
# noter que la dernière année sera exclue, 
# elle donc doit être supérieure à la valeur maximale 
l_50 = list(range(1351, 1802, 50))
l_50[:5],l_50[-5:]

In [ ]:
### fonction pd.cut : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html
# On ajoute une nouvelle colonne qui contient la période sur la base de la liste précédente
# et de la valeur de l'année

cdf_mod['periodes'] = pd.cut(cdf_mod['annee_nais'], l_50, right=False)

### Transformer le code ajouté pour qu'il soit plus lisible
# noter qu'on a arrondi les valeurs
cdf_mod['periodes'] = cdf_mod['periodes'].apply(lambda x : str(int(x.left))+'-'+ str(int(x.right)-1))

# Inspection
cdf_mod.head(3)

In [ ]:
### compter les naissances par périodes de 50 ans
ax = cdf_mod.groupby(by='periodes', observed=True).size().plot(kind='bar',rot=60, fontsize=8, figsize=(16,8))
ax.bar_label(ax.containers[0], fontsize=12)

plt.ylabel('Effectif')
plt.xlabel('Périodes')
plt.title('Naissances par périodes de cinquante ans')
plt.show()

In [ ]:
DActives = cdf_mod[['periodes', 'origine_geog',
 'o1', 'a1', 'o2']].copy(deep=True)

In [ ]:
DActives.groupby(by='periodes', observed=False).size()

In [ ]:
DActives.groupby(by='origine_geog', observed=True).size().sort_values(ascending=False)


In [ ]:
### Structure du tableau à analyser
# nombre de variables
p = DActives.shape[1]
# nombre d'observations
n = DActives.shape[0]
print('Nombre variables:', p, 'Nombre lignes:', n)
# codage en 0/1
X = pd.get_dummies(DActives,prefix='',prefix_sep='')*1


### Distance des individus par rapport au profil moyen

In [ ]:
#Calcul du profil de l'individu moyen
ind_moy = np.sum(X.values,axis=0)/(n*p)
print(ind_moy)

In [ ]:
### Ajouter une colonne avec la distance chi-2 de chaque individu par rapport à l'individu moyen
# pour chaque individu: les individus plus éloignés sont plus rares
X['dist_org'] = X.apply(lambda x: round(np.sum(1/ind_moy*(x/p-ind_moy)**2),4), raw=True, axis=1)

In [ ]:
### Inspecter le tableau
X.sort_values(by='dist_org', ascending=False).head(10)['dist_org']

In [ ]:
### Distribution de la distance chi-2

sns.set_theme(style="whitegrid",rc={"figure.figsize":(10,2)} )


a = X['dist_org']
# a = X['dist_org'][X['dist_org']< 20]

print(a.describe())

ax = sns.violinplot(x=a)

### Noter que au delà des limites les valeurs sont coupées car postulées
ax.set_xlim(left=min(a), right=max(a))

plt.show()

In [ ]:
### Ajouter la distance des individus à leur données
# On créé un nouveau DataFrame qui résulte de la jointure des deux précédents
cdf_mod_dist= pd.merge(cdf_mod, X.dist_org, left_index=True, right_index=True)
cdf_mod_dist[:3]

In [ ]:
### Individus proches du profil moyen
#  donc fréquents
df_filtered = cdf_mod_dist[cdf_mod_dist.dist_org<2]
print(len(df_filtered))
df_filtered.sort_values(by='dist_org')[:5]

In [ ]:
3## Inspecter les combinaisons les plus fréquentes
cdf_mod_dist.groupby(by=['periodes','origine_geog','o1','a1','o2'],\
                       observed=True).size().sort_values(ascending=False).head(10)

In [ ]:
### Individus moyennements distants du profil moyen
df_filtered = cdf_mod_dist[(cdf_mod_dist.dist_org>4.5) & (cdf_mod_dist.dist_org <5.5)]
print(len(df_filtered))
df_filtered.sort_values(by='dist_org')[:5]

In [ ]:
### Individus très distants du profil moyen
#  donc rares, triés en commençant par les plus rares
df_filtered = cdf_mod_dist[cdf_mod_dist.dist_org>15]
print(len(df_filtered))
df_filtered.sort_values(by='dist_org', ascending=False)[:10]

### Inspection des modalités des variables

In [ ]:
### Profil moyen colonnes
moda_moy = np.ones(X.shape[0])/n
moda_moy[:10]


In [ ]:
# enelver la colonne 'dist-org'
df = X.iloc[:,:-1]
# somme en colonne
somme_col = np.sum(df.values,axis=0)
print(somme_col)
# poids des variables_modalités (points modalités)
poids_moda = somme_col/(n*p)
# distance au chi-2 de la valeur moyenne
disto_moda = np.apply_along_axis(arr=df.values/somme_col,axis=0,func1d=lambda x:
np.sum(n*(x-moda_moy)**2))
#np
inertie_moda = poids_moda * disto_moda
#affichage
dfc = pd.DataFrame(np.transpose([poids_moda,disto_moda,inertie_moda]),index=df.columns,columns=['Poids','Disto','Inertie'])


### La disposition des lignes est en fonction des modalités, 
# et la somme des colonnes a été transposée pour avoir des lignes

### Noter que les modalités rares apportent beaucoup de variation
# mais que leur valeur est pondérée en la multipliant par leur (petite) fréquence (=poids)
dfc

In [ ]:
### distance au chi-2 de la valeur moyenne
# Les premières modalités sont celles qui sont plus rares
ax = round(dfc.Disto.sort_values(ascending=True),2).plot(kind='barh', 
                    figsize=(6,10))

ax.bar_label(ax.containers[0])

plt.show()

In [ ]:
### Inertie, i.e. contribution à la variance
ax = round(dfc.Inertie.sort_values(ascending=True),2).plot(kind='barh', 
                    figsize=(6,10))

ax.bar_label(ax.containers[0])

plt.show()

### Représentation graphique

In [ ]:
acm = MCA(row_labels=DActives.index,var_labels=DActives.columns)
acm.fit(DActives.values)

In [ ]:
eig = pd.DataFrame(acm.eig_).transpose()
eig.columns=['contribution','freq','freq_cumulee']

eig.head(), eig.tail()


In [ ]:
### Nombre total de modalités, toute variable confondue
M = X.shape[1]
print('Nombre de modalités:', M)
#nombre max de facteurs
Hmax = M-p
print('Nombre maximal de facteurs:', Hmax)

In [ ]:
### nombre de facteurs calculés par la librairie
print(len(acm.eig_[0]))
### Fréquence cumulée
#acm.eig_[2]

In [ ]:
# https://www.statology.org/pandas-subplots/
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,3))

eig.iloc[:,0].plot(kind='bar', ax=axes[0], title='Eigenvalue des axes')
eig.iloc[:,1].plot(kind='bar', ax=axes[1], title="Proportion de l'eigenvalue ")
eig.iloc[:,2].plot(kind='bar', ax=axes[2], title="Frequence cumulative de l'eigenvalue ")
# Met les valeurs xticks en vertical
fig.autofmt_xdate(rotation=0)
plt.show()

In [ ]:
#éboulis des v.p.

xvalues = len(acm.eig_[0])+1

fix,ax = plt.subplots(figsize=(5,5))
ax.plot(range(1,xvalues),acm.eig_[0],".-")
ax.set_xlabel("Nb. facteurs")
ax.set_ylabel("Val. propres")
plt.title("Eboulis des valeurs propres")
#seuil - Règle de Kaiser
ax.plot([1,xvalues-1],[1/p,1/p],"r--",linewidth=1)
plt.show()

In [ ]:
### Correction de Benzecri, cf. Rakotomalala, Pratique, p.313sqq

#somme en colonne
#récupérer les valeurs propres supérieur à (1/p)
lambada = acm.eig_[0][acm.eig_[0]>1/p]
#print(lambada)

#appliquer la correction
lambada_prim = ((p/(p-1))*(lambada-1/p))**2
#print(lambada_prim)

#faire la somme
S_prim = np.sum(lambada_prim)
#print('u',S_prim)

#et produire les pourcentages
percent_prim = lambada_prim/S_prim*100

#affichage
bzc = pd.DataFrame(np.transpose(np.array([lambada_prim,percent_prim,
                                np.cumsum(percent_prim)])),columns=['Val.P','freq','Cumul_freq'],
                                index=range(1,len(percent_prim)+1))
print(bzc)

In [ ]:
### Préparer un calcul dynamique des dimensions du graphique
# print(acm.col_topandas().columns)
max(acm.col_topandas().col_coord_dim1), min(acm.col_topandas().col_coord_dim1)

In [ ]:
# Mapping des points colonnes

i = 1
dfc = acm.col_topandas()

df_rows = acm.row_topandas()




### avec la correction de Benzecri, 
# 3 axes apportent 95%  de l'information
while i < 8:    

    c1 = 'col_contrib_dim' + str(i)  
    c2 = 'col_contrib_dim' + str(1+i)

    #c1 = 'col_cos2_dim' + str(i)  
    #c2 = 'col_cos2_dim' + str(1+i)


    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,3))

    r1 = dfc[c1].sort_values(ascending=True)[:10]
    r1.plot(kind='barh', ax=axes[0], title="Contribution de l'axe"+ str(i))

    r2 = dfc[c2].sort_values(ascending=True)[:10]
    r2.plot(kind='barh', ax=axes[1], title="Contribution de l'axe"+ str(i+1))

    plt.tight_layout()
    plt.show()


    # rc1 = 'row_contrib_dim' + str(i)  
    # rc2 = 'row_contrib_dim' + str(1+i)
    rc1 = 'row_cos2_dim' + str(i)  
    rc2 = 'row_cos2_dim' + str(1+i)

    ### Filtrer les individus donnant la plus grande contribution à l'axe
    # On retient les individus les mieux représentés sur chaque axe
    rcv1 = df_rows[rc1].sort_values(ascending=False)[:8].index
    rcv2 = df_rows[rc2].sort_values(ascending=False)[:8].index

    lrc = list(set(list(rcv1) + list(rcv2)))


    min_d1 = min(dfc['col_coord_dim' + str(i)])-0.2
    max_d1 = max(dfc['col_coord_dim' + str(i)])+0.2
    min_d2 = min(dfc['col_coord_dim' + str(i+1)])-0.2
    max_d2 = max(dfc['col_coord_dim' + str(i+1)])+0.2

    #représentation simultanée
    #fig,ax = plt.subplots(figsize=(20, 20)
    fig,ax = plt.subplots(figsize=((min_d1*-1 + max_d1)*4, (min_d2*-1 + max_d2)*4))
    ax.axis([min_d1,max_d1,min_d2,max_d2])
    ax.plot([min_d1,max_d1],[0,0],color='silver',linestyle='--')
    ax.plot([0,0],[min_d2,max_d2],color='silver',linestyle='--')

    ax.set_xlabel("Dim."+str(i)+" ("+ str(round(bzc.freq.iloc[(i-1)],2))+")")
    ax.set_ylabel("Dim."+str(i+1)+" ("+ str(round(bzc.freq.iloc[i],2))+")")

    plt.title("Représentation simultanée -" + str(i))
    for i1 in range(df.shape[1]):
        ax.text(acm.col_coord_[i1,(i-1)],acm.col_coord_[i1,i],df.columns[i1],color='dodgerblue')
    
    ### espacer légèrement les individus
    a = -0.08
    for i2 in lrc:
        ax.text(acm.row_coord_[i2,(i-1)],acm.row_coord_[i2,i]+ a,df.index[i2],color='firebrick')        
        a += 0.04
    plt.show()

    print(DActives.iloc[lrc].sort_values(by='periodes').to_markdown())

    i += 2

## MCA de la période après 1801

On commence par inspecter la fréquence des modalités et on exclut les moins fréquentes

In [ ]:
cdf_og = cdf_p[cdf_p.annee_nais > 1800].groupby(by='origine_geog', observed=True).size().sort_values(ascending=False)
cdf_og

In [ ]:
## exclure les modalités moins fréquentes
# Les Balkans impactaient beaucoup l'ACM ci-dessous, donc enlevés

l_og = cdf_og.index.to_list()[:-2]
print(l_og)

In [ ]:
### On notera l'effectif trop important des individus pour lesquels cette variable
# n'est pas renseignée. Mieux vaudra ne pas l'utiliser
cdf_p[cdf_p.annee_nais > 1800].groupby(by='o3').size().sort_values(ascending=False)

In [ ]:
### On procède ici à l'inverse et on retient seulement les individus codés
# à l'eclusion des très faibles effectifs 
df_d3 = cdf_p[cdf_p.annee_nais > 1800].groupby(by='o2').size().sort_values(ascending=False)
df_d3

In [ ]:
## exclure les modalités moins fréquentes
l_d3 = df_d3.index.to_list()[1:-1]
print(l_d3)

In [ ]:
### On agit ici à l'inverse et on retient seulement les individus codés
df_d2 = cdf_p[cdf_p.annee_nais > 1800].groupby(by='a1').size().sort_values(ascending=False)
df_d2

In [ ]:
## exclure les modalités moins fréquentes
l_d2 = df_d2.index.to_list()[1:-1]
print(l_d2)

In [ ]:
cdf_p[cdf_p.annee_nais < 1801].groupby(by='o1').size().sort_values(ascending=False)

### Préparer le tableau à analyser

Il est nécessaire de recoder le tableau pour ne pas avoir des variables avec modalités vides: on utilise donc un nouveau tableau avec les individus nés après 1800.


Aussi, on enlève les modalités les moins fréquentes (cf. ci-dessus). Le bon dosage dépend des questions de recherche.

In [ ]:
### Créer une copie indépendante du DataFrame 
# tout en éliminant les modalités peu fréquentes 
cdf_cont = df_ll[(df_ll['annee_nais'] > 1800)\
                & (df_ll['annee_nais'] < 1976)\
                & (df_ll.origine_geog.isin(l_og))\
                & (df_ll.a1.isin(l_d2))\
                & (df_ll.o2.isin(l_d3))]\
                    .copy(deep=True).reset_index(names='orig_index')

### Noter que le nombre de personnes est réduit
# drastiquement — il faudrait analyser également l'ensemble

print(len(cdf_cont))
cdf_cont.head(3)

In [ ]:
max(cdf_cont.index)

### Codage par période de vingt-cinq ans

In [ ]:
### Année minimale et maximale dans la population
min(cdf_cont['annee_nais']), max(cdf_cont['annee_nais'])

In [ ]:
### Créer une liste d'années pour séparer en périodes de 25 ans
# noter que la dernière année sera exclue, 
# elle donc doit être supérieure à la valeur maximale 
l_25 = list(range(1801, 1977, 25))
l_25[:5],l_25[-5:]

In [ ]:
### fonction pd.cut : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html
# On ajoute une nouvelle colonne qui contient la période sur la base de la liste précédente
# et de la valeur de l'année

cdf_cont['per_25'] = pd.cut(cdf_cont['annee_nais'], l_25, right=False)

### Transformer le code ajouté pour qu'il soit plus lisible
# noter qu'on a arrondi les valeurs
cdf_cont['per_25'] = cdf_cont['per_25'].apply(lambda x : str(int(x.left))+'-'+ str(int(x.right)-1))

# Inspection
cdf_cont.head(3)

In [ ]:
### compter les naissances par périodes de 25 ans
ax = cdf_cont.groupby(by='per_25', observed=True).size().plot(kind='bar',rot=60, fontsize=8, figsize=(16,8))
ax.bar_label(ax.containers[0], fontsize=12)

plt.ylabel('Effectif')
plt.xlabel('Périodes')
plt.title('Naissances par périodes de vingt-cinq ans')
plt.show()

### Population retenue et variables

In [ ]:
DActives_ct = cdf_cont[['per_25', 'origine_geog',
 'o1', 'a1', 'o2']].copy(deep=True)

In [ ]:
DActives_ct.groupby(by='origine_geog', observed=True).size().sort_values(ascending=False)


In [ ]:
DActives_ct.groupby(by='a1', observed=True).size().sort_values(ascending=False)


In [ ]:
DActives_ct.groupby(by='o2', observed=True).size().sort_values(ascending=False)


In [ ]:
#Rcupération des infos - nombre de variables
p = DActives_ct.shape[1]
#nombre d'observations
n = DActives_ct.shape[0]
print('Nombre variables:', p, 'Nombre lignes:', n)
#codage en 0/1
X = pd.get_dummies(DActives_ct,prefix='',prefix_sep='')*1


### Distance des individus par rapport au profil moyen

In [ ]:
#Profil individu moyen
ind_moy = np.sum(X.values,axis=0)/(n*p)
print(ind_moy)

In [ ]:
### Ajouter une colonne avec la distance chi-2 de chaque individu par rapport à l'individu moyen
# pour chaque individu: les individus plus éloignés sont plus rares
X['dist_org'] = X.apply(lambda x: round(np.sum(1/ind_moy*(x/p-ind_moy)**2),4), raw=True, axis=1)

In [ ]:
### Inspecter le tableau
X.sort_values(by='dist_org', ascending=False).head(10)['dist_org']

In [ ]:
### Distribution de la distance chi-2

sns.set_theme(style="whitegrid",rc={"figure.figsize":(10,2)} )


a = X['dist_org']
#a = X['dist_org'][X['dist_org']< 15]

print(a.describe())

ax = sns.violinplot(x=a)

### Noter que au delà des limites les valeurs sont coupées car postulées
ax.set_xlim(left=min(a), right=max(a))

plt.show()

In [ ]:
### Ajouter la distance des individus à leur données
cdf_cont_dist= pd.merge(cdf_cont, X.dist_org, left_index=True, right_index=True)
cdf_cont_dist[:3]

In [ ]:
### Individus proches du profil moyen
#  donc fréquents
df_filtered = cdf_cont_dist[cdf_cont_dist.dist_org<2.2]
print(len(df_filtered))
df_filtered.sort_values(by='dist_org')[:5]

In [ ]:
### Inspecter les combinaisons les plus fréquentes
cdf_cont_dist.groupby(by=['per_25','origine_geog','o1','a1','o2'],\
                       observed=True).size().sort_values(ascending=False).head(10)

In [ ]:
### Individus moyennements distants du profil moyen
df_filtered = cdf_cont_dist[(cdf_cont_dist.dist_org>4.3) & (cdf_mod_dist.dist_org <4.9)]
print(len(df_filtered))
df_filtered.sort_values(by='dist_org')[:5]

In [ ]:
### Individus très distants du profil moyen
#  donc rares, triés en commençant par les plus rares
df_filtered = cdf_cont_dist[cdf_cont_dist.dist_org>15]
print(len(df_filtered))
df_filtered.sort_values(by='dist_org', ascending=False)[:10]

### Inspection des modalités des variables

C'est à ce stade qu'on vérifie le poids des modalités rares et qu'on décide éventuellement d'en enlever encore davantage, afin d'éviter qu'elle cachent la structure du champ scientifique.


Quant aux individus possédant ces modalités rares, on pourra les étudier séparément.


Rappelons que l'ACM appartient au domaine des statistiques descriptives et que pour la recherche en sciences historiques celles-ci ont une fonction heuristique, non explicative. Elles permettent toutefois de mettre en évidence des phénomènes invisibles autrement.

In [ ]:
### Profil moyen colonnes
moda_moy = np.ones(X.shape[0])/n
moda_moy[:10]


In [ ]:
# enelver la colonne 'dist-org'
df = X.iloc[:,:-1]
# somme en colonne
somme_col = np.sum(df.values,axis=0)
print(somme_col)
# poids des variables_modalités (points modalités)
poids_moda = somme_col/(n*p)
# distance au chi-2 de la valeur moyenne
disto_moda = np.apply_along_axis(arr=df.values/somme_col,axis=0,func1d=lambda x:
np.sum(n*(x-moda_moy)**2))
#np
inertie_moda = poids_moda * disto_moda
#affichage
dfc = pd.DataFrame(np.transpose([poids_moda,disto_moda,inertie_moda]),index=df.columns,columns=['Poids','Disto','Inertie'])


### La disposition des lignes est en fonction des modalités, 
# et la somme des colonnes a été transposée pour avoir des lignes
dfc

In [ ]:
### distance au chi-2 de la valeur moyenne
# Les premières modalités sont celles qui sont plus rares
ax = round(dfc.Disto.sort_values(ascending=True),2).plot(kind='barh', 
                    figsize=(6,10))

ax.bar_label(ax.containers[0])

plt.show()

In [ ]:
### Inertie, i.e. contribution à la variance
ax = round(dfc.Inertie.sort_values(ascending=True),2).plot(kind='barh', 
                    figsize=(6,10))

ax.bar_label(ax.containers[0])

plt.show()

### Représentation graphique

In [ ]:
acm = MCA(row_labels=DActives_ct.index,var_labels=DActives_ct.columns)
acm.fit(DActives_ct.values)

In [ ]:
eig = pd.DataFrame(acm.eig_).transpose()
eig.columns=['contribution','freq','freq_cumulee']

eig.head(), eig.tail()


In [ ]:
### Nombre total de modalités, toute variable confondue
M = X.shape[1]
print('Nombre de modalités:', M)
#nombre max de facteurs
Hmax = M-p
print('Nombre maximal de facteurs:', Hmax)

In [ ]:
### nombre de facteurs calculés par la librairie
print(len(acm.eig_[0]))
### Fréquence cumulée
#acm.eig_[2]

In [ ]:
# https://www.statology.org/pandas-subplots/
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,3))

eig.iloc[:,0].plot(kind='bar', ax=axes[0], title='Eigenvalue des axes')
eig.iloc[:,1].plot(kind='bar', ax=axes[1], title="Proportion de l'eigenvalue ")
eig.iloc[:,2].plot(kind='bar', ax=axes[2], title="Frequence cumulative de l'eigenvalue ")
# Met les valeurs xticks en vertical
fig.autofmt_xdate(rotation=0)
plt.show()

In [ ]:
#éboulis des v.p.

xvalues = len(acm.eig_[0])+1

fix,ax = plt.subplots(figsize=(5,5))
ax.plot(range(1,xvalues),acm.eig_[0],".-")
ax.set_xlabel("Nb. facteurs")
ax.set_ylabel("Val. propres")
plt.title("Eboulis des valeurs propres")
#seuil - Règle de Kaiser
ax.plot([1,xvalues-1],[1/p,1/p],"r--",linewidth=1)
plt.show()

*Diagramme d'éboulis*. Représentation graphique ayant pour but d'identifier un point d'inflexion dans une courbe de la variance. Le nom donné à ce type de graphique vient de la ressemblance de la courbe avec le profil des éboulis (scree) au bas d'une falaise. [DataFranca, Diagramme d'éboulis, 2024](https://datafranca.org/wiki/index.php?title=Diagramme_d%27%C3%A9boulis&oldid=93502)

In [ ]:
### Correction de Benzecri, cf. Rakotomalala, Pratique, p.313sqq

#somme en colonne
#récupérer les valeurs propres supérieur à (1/p)
lambada = acm.eig_[0][acm.eig_[0]>1/p]
#print(lambada)

#appliquer la correction
lambada_prim = ((p/(p-1))*(lambada-1/p))**2
#print(lambada_prim)

#faire la somme
S_prim = np.sum(lambada_prim)
#print('u',S_prim)

#et produire les pourcentages
percent_prim = lambada_prim/S_prim*100

#affichage
bzc = pd.DataFrame(np.transpose(np.array([lambada_prim,percent_prim,
                                np.cumsum(percent_prim)])),columns=['Val.P','freq','Cumul_freq'],
                                index=range(1,len(percent_prim)+1))
print(bzc)

In [ ]:
# Mapping des points colonnes

i = 1
dfc = acm.col_topandas()

df_rows = acm.row_topandas()




### avec la correction de Benzecri, 
# 3 axes apportent 95%  de l'information
while i < 6:    

    #c1 = 'col_contrib_dim' + str(i)  
    #c2 = 'col_contrib_dim' + str(1+i)

    c1 = 'col_cos2_dim' + str(i)  
    c2 = 'col_cos2_dim' + str(1+i)


    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,3))

    r1 = dfc[c1].sort_values(ascending=True)[:10]
    r1.plot(kind='barh', ax=axes[0], title="Cos-2 de l'axe"+ str(i))

    r2 = dfc[c2].sort_values(ascending=True)[:10]
    r2.plot(kind='barh', ax=axes[1], title="Cos-2 de l'axe"+ str(i+1))

    plt.tight_layout()
    plt.show()


    # rc1 = 'row_contrib_dim' + str(i)  
    # rc2 = 'row_contrib_dim' + str(1+i)
    rc1 = 'row_cos2_dim' + str(i)  
    rc2 = 'row_cos2_dim' + str(1+i)

    ### Filtrer les individus donnant la plus grande contribution à l'axe
    # On retient les individus les mieux représentés sur chaque axe
    rcv1 = df_rows[rc1].sort_values(ascending=False)[:8].index
    rcv2 = df_rows[rc2].sort_values(ascending=False)[:8].index

    lrc = list(set(list(rcv1) + list(rcv2)))


    min_d1 = min(dfc['col_coord_dim' + str(i)])-0.2
    max_d1 = max(dfc['col_coord_dim' + str(i)])+0.2
    min_d2 = min(dfc['col_coord_dim' + str(i+1)])-0.2
    max_d2 = max(dfc['col_coord_dim' + str(i+1)])+0.2

    #représentation simultanée
    #fig,ax = plt.subplots(figsize=(20, 20))
    fig,ax = plt.subplots(figsize=((min_d1*-1 + max_d1)*5, (min_d2*-1 + max_d2)*5))
    ax.axis([min_d1,max_d1,min_d2,max_d2])
    ax.plot([min_d1,max_d1],[0,0],color='silver',linestyle='--')
    ax.plot([0,0],[min_d2,max_d2],color='silver',linestyle='--')

    ax.set_xlabel("Dim."+str(i)+" ("+ str(round(bzc.freq.iloc[(i-1)],2))+")")
    ax.set_ylabel("Dim."+str(i+1)+" ("+ str(round(bzc.freq.iloc[i],2))+")")

    plt.title("Représentation simultanée -" + str(i))
    for i1 in range(df.shape[1]):
        ax.text(acm.col_coord_[i1,(i-1)],acm.col_coord_[i1,i],df.columns[i1],color='dodgerblue')
    
    ### espacer légèrement les individus
    a = -0.08
    for i2 in lrc:
        ax.text(acm.row_coord_[i2,(i-1)],acm.row_coord_[i2,i]+ a,df.index[i2],color='firebrick')        
        a += 0.04
    plt.show()

    print(DActives_ct.iloc[lrc].sort_values(by='per_25').to_markdown())

    i += 2